In [1]:
from agrifoodpy.pipeline import Pipeline
from datablock_setup import datablock_setup
from pipeline_setup import pipeline_setup

import pandas as pd

In [2]:
advanced_settings_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTanjc08kc5vIN-icUMzMEGA9bJuDesLX8V_u2Ab6zSC4MOhLZ8Jrr18DL9o4ofKIrSq6FsJXhPWu3F/pub?gid=0&single=true&output=csv"

def read_advanced_settings():
    """Reads the advanced settings from the spreadsheet URL"""
    advanced_settings  = pd.read_csv(advanced_settings_url, dtype='string')
    advanced_settings_dict = {}

    for index, row in advanced_settings.iterrows():
        if row["type"] == "float": 
            advanced_settings_dict[row["key"]] = float(row["value"])
        elif row["type"] == "string":
            advanced_settings_dict[row["key"]] = str(row["value"])
        elif row["type"] == "bool":
            advanced_settings_dict[row["key"]] = row["value"] == "TRUE"
    
    return advanced_settings_dict

params = read_advanced_settings()

params["yield_proj"] = 0
params["elasticity"] = 0
params["scaling_nutrient"] = "kCal/cap/day"
params["cereal_scaling"] = True
params["bdleaf_conif_ratio"] = 75

params


{'labmeat_co2e': 2.2,
 'dairy_alternatives_co2e': 0.31,
 'rda_kcal': 2250.0,
 'n_scale': 20.0,
 'bdleaf_seq_ha_yr': 1.3,
 'conif_seq_ha_yr': 11.2,
 'peatland_seq_ha_yr': 20.0,
 'managed_arable_seq_ha_yr': 2.0,
 'managed_pasture_seq_ha_yr': 2.0,
 'mixed_farming_seq_ha_yr': 2.0,
 'beccs_crops_seq_ha_yr': 23.5,
 'dairy_herd_grazing': 0.05,
 'dairy_herd_beef': 0.52,
 'horticulture_land_ratio': 0.086,
 'pulse_land_ratio': 0.033,
 'mixed_farming_production_scale': 0.93,
 'mixed_farming_secondary_production_scale': 0.9,
 'agroecology_tree_coverage': 0.1,
 'nitrogen_prod_factor': 0.0,
 'nitrogen_ghg_factor': 0.1,
 'manure_prod_factor': 0.0,
 'manure_ghg_factor': 0.07,
 'breeding_prod_factor': 0.0,
 'breeding_ghg_factor': 0.08,
 'methane_prod_factor': 0.0,
 'methane_ghg_factor': 0.13,
 'fossil_livestock_prod_factor': 0.0,
 'fossil_livestock_ghg_factor': 0.1,
 'fossil_arable_prod_factor': 0.0,
 'fossil_arable_ghg_factor': 0.1,
 'scaling_nutrient': 'kCal/cap/day',
 'emission_factors': 'NDC 2020',

In [3]:
params["ruminant"] = 0
params["pig_poultry"] = 0
params["fish_seafood"] = 0
params["dairy"] = 0
params["eggs"] = 0
params["fruit_veg"] = 0
params["pulses"] = 0
params["meat_alternatives"] = 0
params["dairy_alternatives"] = 0
params["waste"] = 0
params["foresting_pasture"] = 0
params["land_BECCS"] = 0
params["horticulture"] = 0
params["pulse_production"] = 0
params["lowland_peatland"] = 0
params["upland_peatland"] = 0
params["pasture_soil_carbon"] = 0
params["arable_soil_carbon"] = 0
params["mixed_farming"] = 0
params["silvopasture"] = 0
params["nitrogen"] = 0
params["methane_inhibitor"] = 0
params["stock_density"] = 0
params["manure_management"] = 0
params["animal_breeding"] = 0
params["fossil_livestock"] = 0
params["agroforestry"] = 0
params["vertical_farming"] = 0
params["fossil_arable"] = 0
params["waste_BECCS"] = 0
params["overseas_BECCS"] = 0
params["DACCS"] = 0

In [4]:
food_system = Pipeline(datablock_setup())
food_system = pipeline_setup(food_system, params)
food_system.run()
datablock_result = food_system.datablock

datablock_result

{'food': {'1000 T/year': <xarray.Dataset> Size: 34kB
  Dimensions:      (Year: 1, Item: 97)
  Coordinates:
      Region       int64 8B 229
      Region_name  <U52 208B 'United Kingdom of Great Britain and Northern Irel...
    * Year         (Year) int64 8B 2020
    * Item         (Item) int64 776B 2511 2513 2514 2515 ... 2782 2807 2948 2949
      Item_name    (Item) <U31 12kB 'Wheat and products' ... 'Eggs'
      Item_group   (Item) <U24 9kB 'Cereals - Excluding Beer' ... 'Eggs'
      Item_origin  (Item) <U16 6kB 'Vegetal Products' ... 'Animal Products'
  Data variables: (12/13)
      stock        (Year, Item) float32 388B -3.265e+03 46.0 200.0 ... 4.0 5.0 0.0
      losses       (Year, Item) float32 388B 48.0 41.0 27.0 2.0 ... nan nan 9.0
      processing   (Year, Item) float32 388B 336.0 1.297e+03 ... 3.741e+03 nan
      food         (Year, Item) float32 388B 6.97e+03 207.0 ... 1.345e+04 762.0
      other        (Year, Item) float32 388B 841.0 nan 444.0 nan ... 59.0 0.0 0.0
      resi